# Scanning an Index with Pytesseract

## Part 1

In [58]:
import pytesseract
from PIL import Image

In [59]:
image_file = "images/index_02.JPG"
img = Image.open(image_file)

In [60]:
ocr_result = pytesseract.image_to_string(img)

In [61]:
print(ocr_result)

INDEX NOMINUM.

SCRIPSIT KAROLUS HAMPE.

Maior numerus indicat paginam, minor lineam.

A. v. E.

Aaron 38, 20. 123, 18. 168, 6. 203, 38. |

Abacuc propheta 36,5. 319,29. 373, 23.
Abagarus rex Edessenorum 382, 13.
Abdenago 585, 3.

Abel 26, 11.

Abigail 606, 6.

Abraham 26,18. 124,18. 133, 23. 2:
264. 302, 13. 303, 13. 320, 40. 328, 1.
496, 22.

Absalon 102, 5.

Acgfredi v. Ecgfridus.

Achab rex 504, 1.

Achaz rex 504,1.

Achiel 606, 19.

Achilon 496, 26.

Adal-, Adel-, Adhel-, Aedel-, Aedel-,
‘Aedil-, Aed
Athel-, Athil-, Aethel-, Aethil-,
Ethel-.

Aedelbaldusabb.S. Petri Wiremuthen-
sis et S. Pauli Girwensis 110, 28.

Aedilberctus, Aedilberthus ep. Ha-
gulstadensis, antea ep. Candidae
casae sive Witernensis 27,43. 72,16.

Adalbertus, Aedilberctus sive Magus,
discipulus et capellanus Arnonis
archiep. Salisburg., postea_abb.
Ferrariensis 254, 1. 320, 41. 322, 19.
416. 418, 14. 422, 10.

Adalpertabb. [Tegernseensis ?]497,23.

Adalbertus presb. monasterii 8. Mar-
tini Turonensis 399, 3.

A

## Part 2

In [62]:
import pytesseract
import cv2

In [63]:
image = cv2.imread("images/index_02.JPG")
base_iamge = image.copy()

In [64]:
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [65]:
cv2.imwrite("images/results 2/index_gray.png", gray)

True

In [66]:
blur = cv2.GaussianBlur(gray, (7,7), 0)

In [67]:
cv2.imwrite("images/results 2/index_blur.png", blur)

True

In [68]:
# Create a threshold image
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [69]:
cv2.imwrite("images/results 2/index_thresh.png", thresh)

True

In [70]:
# Individual creation of kernels and dialiation
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,13))

In [71]:
cv2.imwrite("images/results 2/index_kernel.png", kernel)

True

In [72]:
dilate = cv2.dilate(thresh, kernel, iterations = 1)

In [73]:
cv2.imwrite("images/results 2/index_dilate.png", dilate)

True

In [74]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [75]:
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

In [76]:
cnts = sorted(cnts, key = lambda x: cv2.boundingRect(x)[0])

In [77]:
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if h > 200 and w >20:
        roi = image[y:y+h, x:x+w]
        cv2.imwrite("images/results 2/index_roi.png", roi)
        cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
cv2.imwrite("images/results 2/index_bbox.png", image)

True

In [79]:
results = []
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    if h > 200 and w >20:
        roi = image[y:y+h, x:x+w]
        cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
        ocr_result = pytesseract.image_to_string(roi)
        ocr_result = ocr_result.split("\n")
        for item in ocr_result:
            results.append(item)
#cv2.imwrite("images/results 2/index_bbox.png", image)

In [80]:
print(results)

[' ', '', 'A. v. E.', '', 'Aaron 38, 20. 123, 18. 168, 6. 203, 38.', 'Abacuc propheta 36,5. 319,29. 373, 23.', 'Abagarus rex Edessenorum 382, 13.', 'Abdenago 585, 3.', 'Abel 26, 11.', 'Abigail 606, 6.', 'Abraham 26,18. 124,18. 133, 23.', '264. 302, 13. 303, 13. 320, 40.', '496, 22.', 'Absalon 102, 5.', 'Acgfredi v. Ecgtridus.', 'Achab rex 504, 1.', 'Achaz rex 504,1.', 'Achiel 606, 19.', 'Achilon 496, 26.', 'Adal-, Adel-, Adhel-, Aedel-, Aed', '', 'Aedil-, Aedil- Edel-, Edil-, Athal-,', 'Athel-, Athil-, Aethel-, Aethil-,', 'Ethel-.', 'Aedelbaldusabb.S. Petri Wiremuthen-', 'sis et S. Pauli Girwensis 110, 28.', 'Aedilberctus, Aedilberthus ep. Ha-', 'gulstadensis, antea ep. Candidae', 'casae sive Witernensis 27,43. 72,16.', 'Adalbertus, Aedilberctus sive Magus,', 'discipulus et capellanus Arnonis', 'archiep. Salisburg., postea_abb.', 'Ferrariensis 254, 1. 320, 41. 322, 19.', '416. 418,14. 422, 10.', 'Adalpertabb.[Tegernseensis ?]497,23.', 'Adalbertus presb. monasterii S. Mar-', 'tini Turon

In [91]:
entities = []
for item in results:
    item = item.strip().replace("\n","")
    item = item.split(" ")[0]
    if len(item) > 2:
        if item[0].isupper() and item[0] == 'A' and "-" not in item:
            item = item.split(".")[0].replace(",","").replace(";","")
            entities.append(item)

In [92]:
print(entities)

['Aaron', 'Abacuc', 'Abagarus', 'Abdenago', 'Abel', 'Abigail', 'Abraham', 'Absalon', 'Acgfredi', 'Achab', 'Achaz', 'Achiel', 'Achilon', 'Aedelbaldusabb', 'Aedilberctus', 'Adalbertus', 'Adalpertabb', 'Adalbertus', 'Aedilburga', 'Aethelfleda', 'Adalgarius', 'Adelgisa', 'Adalgisus', 'Aethelhardus', 'Aedelhardus', 'Adalhardus', 'Adalmondus', 'Adelperga', 'Adalpert', 'Aedelredus', 'Aethilredus', 'Aedilthyda', 'Adhelricus', 'Aedilthyda', 'Aedelwaldus', 'Aedilthyda', 'Adalwinus', 'Aethiluuinus', 'Adam', 'Adam', 'Adaula', 'Adoredus', 'Adrianus', 'Adriaticum', 'Aegesippus', 'Aegil', 'Aegyptus', 'Aegyptii', 'Aegyptiacascola', 'Aeine', 'Aelberhtus', 'Aelfvaldus', 'Aelim', 'Aeneades', 'Aesculapius', 'Aesne', 'Aethiopia', 'Africa', 'Africanus', 'Agar', 'Agareni', 'Agathensis', 'Aginus']


In [94]:
entities = list(set(entities))
entities.sort()
print (entities)

['Aaron', 'Abacuc', 'Abagarus', 'Abdenago', 'Abel', 'Abigail', 'Abraham', 'Absalon', 'Acgfredi', 'Achab', 'Achaz', 'Achiel', 'Achilon', 'Adalbertus', 'Adalgarius', 'Adalgisus', 'Adalhardus', 'Adalmondus', 'Adalpert', 'Adalpertabb', 'Adalwinus', 'Adam', 'Adaula', 'Adelgisa', 'Adelperga', 'Adhelricus', 'Adoredus', 'Adrianus', 'Adriaticum', 'Aedelbaldusabb', 'Aedelhardus', 'Aedelredus', 'Aedelwaldus', 'Aedilberctus', 'Aedilburga', 'Aedilthyda', 'Aegesippus', 'Aegil', 'Aegyptiacascola', 'Aegyptii', 'Aegyptus', 'Aeine', 'Aelberhtus', 'Aelfvaldus', 'Aelim', 'Aeneades', 'Aesculapius', 'Aesne', 'Aethelfleda', 'Aethelhardus', 'Aethilredus', 'Aethiluuinus', 'Aethiopia', 'Africa', 'Africanus', 'Agar', 'Agareni', 'Agathensis', 'Aginus']
